In [87]:

#imports de procesado
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
#imports del modelo a usar
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_val_score,train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, roc_auc_score,classification_report
pd.set_option('display.max_rows', None)
from joblib import dump
label_encoder = LabelEncoder()

In [88]:
train = pd.read_csv("../data/processed/train_processed.csv")
df_test = pd.read_csv("../data/processed/test_processed.csv")

In [89]:
#Parecen estar bien, no hay columnas residuales.
df_test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Spa_VRDeck_0.0,all_0.0,Earth_0.0,TRAPPIST-1e_0.0
0,0,0,True,2784,2,27.0,False,0.0,0.0,0.0,0.0,0.0,2912,True,True,False,False
1,1,0,False,1867,2,19.0,False,0.0,9.0,0.0,2823.0,0.0,2406,False,False,True,True
2,2,1,True,257,0,31.0,False,0.0,0.0,0.0,0.0,0.0,3376,True,True,False,False
3,3,1,False,259,2,38.0,False,0.0,6652.0,0.0,181.0,585.0,2711,False,False,False,True
4,4,0,False,1940,2,20.0,False,10.0,0.0,635.0,0.0,0.0,668,True,False,True,True


In [90]:
#No los quite antes, los quito ahora, igual quiero tener la test intacta por si acaso.
test = df_test.drop(["Name", "PassengerId","Cabin"], axis =1)

In [91]:
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Spa_VRDeck_0.0,all_0.0,Earth_0.0,TRAPPIST-1e_0.0
0,0,True,2,27.0,False,0.0,0.0,0.0,0.0,0.0,True,True,False,False
1,0,False,2,19.0,False,0.0,9.0,0.0,2823.0,0.0,False,False,True,True
2,1,True,0,31.0,False,0.0,0.0,0.0,0.0,0.0,True,True,False,False
3,1,False,2,38.0,False,0.0,6652.0,0.0,181.0,585.0,False,False,False,True
4,0,False,2,20.0,False,10.0,0.0,635.0,0.0,0.0,True,False,True,True


In [92]:
X_train, X_test, y_train, y_test = train_test_split(train.drop(["Transported"],axis=1), train["Transported"], test_size=0.2, random_state=42)

In [94]:
#Aqui la pipeline para buscar el modelo mas optimo, simplemente eso.
#Primero buscare cual es el mejor modelo y a que precision llega.
pipe = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("classifier", RandomForestClassifier())
])

log_params = {
    'selectkbest__k':np.arange(5,15),
    'classifier': [LogisticRegression()],
    'classifier__C': [0.1,1,10]
}
rf_params = {
    'scaler': [StandardScaler(), None],
    'selectkbest__k':np.arange(5,15),
    'classifier': [RandomForestClassifier()],
    'classifier__max_depth': [3,5,7]
}
gb_params = {
    'scaler': [StandardScaler(), None],
    'selectkbest__k':np.arange(5,15),
    'classifier': [GradientBoostingClassifier()],
    'classifier__max_depth': [3,5,7]
}
knn_params = {
    'selectkbest__k':np.arange(5,15),
    'classifier': [KNeighborsClassifier()],
    'classifier__n_neighbors': np.arange(5,15)
}
svm_params = {
    'selectkbest__k':np.arange(5,15),
    'classifier': [SVC()],
    'classifier__C': [0.1,1,10]
}

search_space = [
    log_params,
    rf_params,
    gb_params,
    knn_params,
    svm_params   
]


In [95]:
#aqui ejecutamos la pipeline
clf_gs = GridSearchCV(estimator=pipe, param_grid=search_space, cv=3, scoring="accuracy", verbose=3, n_jobs=-1)

In [96]:
#Solo me interesaba para sacar el mejor modelo, como es random tree lo dejo comentado.
#clf_gs.fit(X_train, y_train)

In [1]:
"""modelo = clf_gs.best_estimator_
mhiperparametros = clf_gs.best_params_
rendimiento = clf_gs.best_score_

pred = modelo.predict(test)

scores = cross_val_score(modelo, X_train, y_train, cv=5, scoring='accuracy')
print(f'Accuracy en validación cruzada: {scores.mean()} (Desviación estándar: {scores.std()})')""";

In [98]:
#ahora que tenemos el mejor moelo, paso directamente a darle un arbol mas cargado, a ver si mejora.
rf_pipeline2 = Pipeline([
    ('scaler', StandardScaler()),
    ('selectkbest', SelectKBest()),
    ('classifier', RandomForestClassifier()),
])


rf_params2 = {
    'scaler': [StandardScaler(), None],
    'selectkbest__k': np.arange(5, 15),
    'classifier__n_estimators': [50, 100, 200],
    'classifier__max_depth': [3, 5, 7],
    'classifier__min_samples_split': [2, 5, 10],
}


grid_search2 = GridSearchCV(rf_pipeline2, rf_params2, cv=5, scoring='accuracy', n_jobs=-1)


grid_search2.fit(X_train, y_train)


print("Mejores Parámetros:", grid_search2.best_params_)
print("Mejor Accuracy en Validación Cruzada:", grid_search2.best_score_)

Mejores Parámetros: {'classifier__max_depth': 7, 'classifier__min_samples_split': 10, 'classifier__n_estimators': 50, 'scaler': None, 'selectkbest__k': 14}
Mejor Accuracy en Validación Cruzada: 0.7960905926588707


In [99]:
modelo2 = grid_search2.best_estimator_
mhiperparametros2 = grid_search2.best_params_
rendimiento2 = grid_search2.best_score_

In [100]:
pred2 = modelo2.predict(test)

In [101]:
#79% de precision, no esta mal.
scores2 = cross_val_score(modelo2, X_train, y_train, cv=5, scoring='accuracy')
print(f'Accuracy en validación cruzada: {scores2.mean()} (Desviación estándar: {scores2.std()})')

Accuracy en validación cruzada: 0.7916326435616424 (Desviación estándar: 0.006846622756444476)


In [102]:
test.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Spa_VRDeck_0.0,all_0.0,Earth_0.0,TRAPPIST-1e_0.0
0,0,True,2,27.0,False,0.0,0.0,0.0,0.0,0.0,True,True,False,False
1,0,False,2,19.0,False,0.0,9.0,0.0,2823.0,0.0,False,False,True,True
2,1,True,0,31.0,False,0.0,0.0,0.0,0.0,0.0,True,True,False,False
3,1,False,2,38.0,False,0.0,6652.0,0.0,181.0,585.0,False,False,False,True
4,0,False,2,20.0,False,10.0,0.0,635.0,0.0,0.0,True,False,True,True


In [105]:
pasajero = pd.read_csv("../data/raw/test.csv")
pasajero.head()
#lo guardamos con el formato que pide kaggle.
submission = pd.DataFrame()
submission["PassengerId"] = pasajero["PassengerId"]
submission["Transported"] = pred2

In [106]:
#esta perfecto, lo guardamos asi.
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [107]:
submission.to_csv("../data/processed/Submission.csv", index=False)

In [109]:
#por el momento guardamos el modelo por si queremos re-entrenarlo.
modelo_entrenado = modelo2


dump(modelo_entrenado, '../Modelos/Arbol_aleatorio.joblib')

['../Modelos/Arbol_aleatorio.joblib']